In [13]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import OneSidedSelection

In [14]:
import pandas as pd
import numpy as np
path='dataset_full.csv'
data = pd.read_csv(path,encoding="ISO-8859-1")
y = data['phishing']
X = data.drop('phishing',axis=1)
X = X[['qty_dot_url', 'qty_hyphen_url', 'qty_underline_url', 'qty_slash_url',
       'qty_equal_url', 'qty_at_url', 'qty_and_url', 'qty_tilde_url',
       'qty_plus_url', 'qty_percent_url', 'qty_tld_url', 'length_url',
       'qty_dot_domain', 'qty_hyphen_domain', 'qty_vowels_domain',
       'domain_length', 'domain_in_ip', 'server_client_domain',
       'qty_dot_directory', 'qty_hyphen_directory', 'qty_underline_directory',
       'qty_slash_directory', 'qty_equal_directory', 'qty_at_directory',
       'qty_and_directory', 'qty_exclamation_directory', 'qty_space_directory',
       'qty_comma_directory', 'qty_plus_directory', 'qty_asterisk_directory',
       'qty_hashtag_directory', 'qty_dollar_directory',
       'qty_percent_directory', 'directory_length', 'qty_dot_file',
       'qty_hyphen_file', 'qty_underline_file', 'qty_slash_file',
       'qty_questionmark_file', 'qty_equal_file', 'qty_and_file',
       'qty_exclamation_file', 'qty_space_file', 'qty_comma_file',
       'qty_plus_file', 'qty_percent_file', 'file_length', 'qty_dot_params',
       'qty_hyphen_params', 'qty_underline_params', 'qty_slash_params',
       'qty_questionmark_params', 'qty_equal_params', 'qty_at_params',
       'qty_and_params', 'qty_exclamation_params', 'qty_space_params',
       'qty_tilde_params', 'qty_comma_params', 'qty_plus_params',
       'qty_asterisk_params', 'qty_hashtag_params', 'qty_dollar_params',
       'qty_percent_params', 'params_length', 'tld_present_params',
       'qty_params', 'time_response', 'domain_spf', 'asn_ip',
       'time_domain_activation', 'time_domain_expiration', 'qty_ip_resolved',
       'qty_nameservers', 'qty_mx_servers', 'ttl_hostname',
       'tls_ssl_certificate', 'qty_redirects', 'url_google_index',
       'url_shortened']]
print (X.shape)
print( y.shape)

(88647, 80)
(88647,)


In [15]:
from collections import Counter
counter = Counter(y)
print(counter)

Counter({0: 58000, 1: 30647})


In [16]:
# First apply ADASYN to slightly oversample the minority class
ada = ADASYN(sampling_strategy=0.67, random_state=42)  # This will create roughly 2/3 of majority class
X_res, y_res = ada.fit_resample(X, y)

# Then apply OneSidedSelection with modified parameters
undersample = OneSidedSelection(n_neighbors=1, n_seeds_S=300)
X_res1, y_res1 = undersample.fit_resample(X_res, y_res)

#summarize the new class distribution
print('Resampled dataset shape %s' % Counter(y_res1))

Resampled dataset shape Counter({0: 45438, 1: 39246})


In [17]:
from collections import Counter
counter = Counter(y)
print(counter)

Counter({0: 58000, 1: 30647})


In [18]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import ADASYN
print('Original dataset shape %s' % Counter(y))
ada = ADASYN(sampling_strategy=0.67,random_state=42)
X_res, y_res = ada.fit_resample(X, y)
#define the undersampling method
undersample = OneSidedSelection(n_neighbors=1, n_seeds_S=200)
#transform the dataset
X_res1, y_res1 = undersample.fit_resample(X_res, y_res)
print('Resampled dataset shape %s' % Counter(y_res1))

Original dataset shape Counter({0: 58000, 1: 30647})
Resampled dataset shape Counter({0: 47592, 1: 39246})


In [19]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report,confusion_matrix
scaler = StandardScaler()
scaler.fit(data.drop('phishing',axis=1))
scaled_features = scaler.transform(data.drop('phishing',axis=1))
df_feat = pd.DataFrame(scaled_features,columns=data.columns[:-1])
df_feat.head()
X_train, X_test, y_train, y_test = train_test_split(scaled_features,data['phishing'],
                test_size=0.30, random_state=42)
knn1 = KNeighborsClassifier(n_neighbors=3)
#X_train_res, y_train_res = ada.fit_resample(X_train, y_train.ravel())
ada = ADASYN(sampling_strategy=0.67, random_state=42)
undersample = OneSidedSelection(n_neighbors=1, n_seeds_S=300)
X_train_res, y_train_res = ada.fit_resample(X_train, y_train.ravel())
X_train_res1, y_train_res1 = undersample.fit_resample(X_train_res, y_train_res)
knn1.fit(X_train_res1,y_train_res1.ravel())
pred = knn1.predict(X_test)

print('WITH K=3 and after balancing')
print('\n')
print(confusion_matrix(y_test,pred))
print('\n')
print(classification_report(y_test,pred))

C:\Users\SY\AppData\Local\Temp\ipykernel_42752\134275641.py:16: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  X_train_res, y_train_res = ada.fit_resample(X_train, y_train.ravel())


WITH K=3 and after balancing


[[16300  1149]
 [  380  8766]]


              precision    recall  f1-score   support

           0       0.98      0.93      0.96     17449
           1       0.88      0.96      0.92      9146

    accuracy                           0.94     26595
   macro avg       0.93      0.95      0.94     26595
weighted avg       0.95      0.94      0.94     26595



In [20]:
y_pred_proba = knn1.predict_proba(X_test)[:,1]

In [21]:
from sklearn.metrics import roc_curve

In [22]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

In [23]:
#Area under ROC curve
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,y_pred_proba)

np.float64(0.9699977230196597)

In [24]:
from sklearn.ensemble import RandomForestClassifier

from sklearn import model_selection
# random forest model creation
rfc = RandomForestClassifier()
rfc.fit(X_train_res1,y_train_res1)
# predictions
rfc_predict = rfc.predict(X_test)
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
rfc_cv_score = cross_val_score(rfc, X, y, cv=10, scoring='roc_auc')
rfc_cv_score = cross_val_score(rfc, X, y, cv=10, scoring='roc_auc')
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, rfc_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, rfc_predict))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

=== Confusion Matrix ===
[[16894   555]
 [  313  8833]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.98      0.97      0.97     17449
           1       0.94      0.97      0.95      9146

    accuracy                           0.97     26595
   macro avg       0.96      0.97      0.96     26595
weighted avg       0.97      0.97      0.97     26595



=== All AUC Scores ===
[0.99600419 0.99516417 0.99354323 0.99513723 0.99406387 0.99575384
 0.99576447 0.9960068  0.99554073 0.99496241]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.9951940949410456
